In [12]:
import csv
import itertools
import os
import pickle

import tqdm

import protein
from protein import Protein, trypsin, pepsin, OXIDATION_MET, Peptide

import measurement
from measurement import read_mgf

import pandas as pd
import numpy as np

from multiprocessing import Pool

import importlib
from pprint import pprint

from pyteomics import mzid

importlib.reload(measurement)
importlib.reload(protein)


<module 'protein' from '/Users/eugen/code/bp/bp-code/src/protein.py'>

In [2]:
MYST = "KKLTKEGAAALCKMKHLADKVAKERSQELKDRTQNFAGYIEFELYRIDYWLEKLNGPKGRKDGYAKLSDSDIEKVKEIFNKAKDGITKQLPEAKKAGEEAGKLHTEVKKAAENARGQDLDDDTAKSTGLYRVLNWYCITKEERHNATPNCDGIQFRKHYLSVNRSAIDCSSTSYEENYDWSANALQVALNSWEDVKPKKLESAGSDKNCNIGQSSESHPCTMTEEWQTPYKETVEKLRELEDAYQRGKKAHDAMLGYANTAYAVNTKVEQEKPLTTGLEVLFQGPSAEPEA"
OVA = "GSIGAASMEFCFDVFKELKVHHANENIFYCPIAIMSALAMVYLGAKDSTRTQINKVVRFDKLPGFGDSIEAQCGTSVNVHSSLRDILNQITKPNDVYSFSLASRLYAEERYPILPEYLQCVKELYRGGLEPINFQTAADQARELINSWVESQTNGIIRNVLQPSSVDSQTAMVLVNAIVFKGLWEKAFKDEDTQAMPFRVTEQESKPVQMMYQIGLFRVASMASEKMKILELPFASGTMSMLVLLPDEVSGLEQLESIINFEKLTEWTSSNVMEERKIKVYLPRMKMEEKYNLTSVLMAMGITDVFSSSANLSGISSAESLKISQAVHAAHAEINEAGREVVGSAEAGVDAASVSEEFRADHPFLFCIKHIATNAVLFFGRCVSP"
LYS = "KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL"
BSA = "DTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLTSSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA"
# LYS můstky
# 6,127
# VFGRCELAAA + WIRGCRL

# 30,115
# GNWVCAAKFE + WRNRCKGTDV

# 64,80
# SRWWCNDGRT + CNIPCSALLS
# WWCNDGR+NLCNIPCSALLSSDITASVNCAK

# 76,94
# SRNLCNIPCS + ASVNCAKKIV

# TODO
# Protein Pilot
# Paragon

# AA tagy ze spektra

In [3]:
measurements = list(read_mgf("../data/mgf/190318_LYS_AT_50x_05.mgf"))

In [5]:
[(LYS[b:e], (b, e)) for b, e in trypsin(LYS)]

[('K', (0, 1)),
 ('VFGR', (1, 5)),
 ('CELAAAMK', (5, 13)),
 ('R', (13, 14)),
 ('HGLDNYR', (14, 21)),
 ('GYSLGNWVCAAK', (21, 33)),
 ('FESNFNTQATNR', (33, 45)),
 ('NTDGSTDYGILQINSR', (45, 61)),
 ('WWCNDGR', (61, 68)),
 ('TPGSR', (68, 73)),
 ('NLCNIPCSALLSSDITASVNCAK', (73, 96)),
 ('K', (96, 97)),
 ('IVSDGNGMNAWVAWR', (97, 112)),
 ('NR', (112, 114)),
 ('CK', (114, 116)),
 ('GTDVQAWIR', (116, 125)),
 ('GCR', (125, 128)),
 ('L', (128, 129))]

In [13]:
protein = Protein(LYS)
peptides = list(protein.digest(trypsin, charges={1, 2, 3, 4, 5, 6}, mc=7))

In [14]:
peptides

 ...]

In [24]:
# Determines how many matches we can get on precursor weight alone

import time

tolerance_ppm_precursor = 15
peps_with_threshold = {}
for p in peptides:
    t = (tolerance_ppm_precursor / 1e6) * p.total_mz
    peps_with_threshold.setdefault(p.total_charge, []).append((p, t))

result = []
start_time = time.time()
for i, m in enumerate(measurements):
    for pepid, (pep, t) in enumerate(peps_with_threshold[m.charge]):
        if abs(pep.total_mz - m.peptide_mz) <= t:
            result.append(
                (
                    m,
                    pep,
                )
            )
end_time = time.time()

print(f"One measurement takes about {(end_time - start_time) / len(measurements)} ms")

One measurement takes about 0.4704053526897539 ms


In [5]:

def for_martin(path, prot, enzyme, maxskip=5):
    with open(path, "w") as file:
        p = Protein(prot)
        peptides_initial = enzyme(p.sequence)
        peptides_skip = {
            (peptides_initial[i][0], peptides_initial[i + s][1], s)
            for s in range(maxskip + 1)
            for i in range(len(peptides_initial) - s)
        }

        def variants(peptide):
            seq = p.sequence[peptide[0] : peptide[1]]
            return [
                {
                    "charge": 1,
                    "seq": seq,
                    "mods": mods,
                    "beg": peptide[0],
                    "end": peptide[1],
                    "mc": peptide[2],
                }
                for mods in OXIDATION_MET.somewhere(seq)
            ]

        cysteines = [i for i, aa in enumerate(p.sequence) if aa == "C"]
        peptide_variants = {p: variants(p) for p in peptides_skip}
        peptides_having_cysteine = {
            c: [p for p in peptides_skip if p[0] <= c < p[1]] for c in cysteines
        }
        peptides_joined = [
            {
                "peps": [v],
                "cysteines": [],
            }
            for pepvars in peptide_variants.values()
            for v in pepvars
        ]

        for beg, end in itertools.combinations(cysteines, 2):
            for p1 in peptides_having_cysteine[beg]:
                for p2 in peptides_having_cysteine[end]:
                    for v1 in peptide_variants[p1]:
                        for v2 in peptide_variants[p2]:
                            if max(v1["beg"], v2["beg"]) <= min(v1["end"], v2["end"]):
                                peptides_joined.append(
                                    {
                                        "peps": [v1, v2],
                                        "cysteines": [(beg, end)],
                                    }
                                )

        writer = csv.DictWriter(
            file,
            fieldnames=[
                "peptide",
                "mc",
                "modifications",
                "mass",
                "connected_cysteines",
            ],
        )
        writer.writeheader()

        peptides = [Peptide(**pepvars) for pepvars in peptides_joined]

        for pep in peptides:
            if "C" in pep:
                writer.writerow(
                    {
                        "peptide": str(pep),
                        "mc": ",".join(str(mc) for mc in pep.mcs),
                        "modifications": pep.modstr,
                        "mass": pep.total_mass,
                        "connected_cysteines": ",".join(
                            "+".join(str(c) for c in cs) for cs in pep.cysteines
                        ),
                    }
                )


for protein, name in [(LYS, "lys"), (BSA, "bsa"), (MYST, "hagen")]:
    for enzyme, enzyme_name in [(trypsin, "try"), (pepsin, "pep")]:
        for_martin(
            f"../out/martin/peptides_mc7_{name}_{enzyme_name}.csv",
            protein,
            enzyme,
            maxskip=7,
        )


In [55]:
counts = {}

for (m, p) in result:
    if len(p.peptides) == 1 and p.modstr == "":
        counts.setdefault(m.scan, []).append(p)

for scan, peptides in counts.items():
    print(f"{scan}: {peptides[0]}")

846: RHGLDNYR
917: HGLDNYR
1015: HGLDNYR
1070: HGLDNYR
1122: HGLDNYR
1187: HGLDNYR
1259: HGLDNYR
1335: HGLDNYR
1417: HGLDNYR
1474: FESNFNTQATNR
1505: HGLDNYR
1604: HGLDNYR
1693: HGLDNYR
1783: HGLDNYR
1902: HGLDNYR
1997: FESNFNTQATNR
2022: FESNFNTQATNR
2030: HGLDNYR
2107: FESNFNTQATNR
2136: FESNFNTQATNR
2149: HGLDNYR
2206: FESNFNTQATNR
2281: FESNFNTQATNR
2284: HGLDNYR
2323: FESNFNTQATNR
2392: FESNFNTQATNR
2393: HGLDNYR
2471: NTDGSTDYGILQINSR
2493: FESNFNTQATNR
2530: FESNFNTQATNR
2532: HGLDNYR
2635: FESNFNTQATNR
2656: FESNFNTQATNR
2658: HGLDNYR
2770: FESNFNTQATNR
2773: HGLDNYR
2774: FESNFNTQATNR
2909: FESNFNTQATNR
2912: HGLDNYR
2914: FESNFNTQATNR
3032: FESNFNTQATNR
3036: HGLDNYR
3037: FESNFNTQATNR
3104: NTDGSTDYGILQINSR
3175: FESNFNTQATNR
3210: FESNFNTQATNR
3211: HGLDNYR
3303: FESNFNTQATNR
3338: HGLDNYR
3342: FESNFNTQATNR
3417: FESNFNTQATNR
3456: HGLDNYR
3463: FESNFNTQATNR
3529: FESNFNTQATNR
3540: GTDVQAWIR
3564: HGLDNYR
3570: FESNFNTQATNR
3626: FESNFNTQATNR
3710: HGLDNYR
3714: FESNFNTQA

In [66]:
str(peptides[0])

'FESNFNTQATNR'

In [36]:
count_76_94 = 0

for (m, p) in result:
    if len(p.peptides) == 1 and "NLCNIPCSALLSSDITASVNCAK" in str(p):
        count_76_94 += 1

count_76_94

9

In [34]:
counts_cysteines = {}
for (m, p) in result:
    if len(p.peptides) > 1:
        counts_cysteines.setdefault(p.cysteines[0], 0)
        counts_cysteines[p.cysteines[0]] += 1

{(5, 126): 828,
 (5, 114): 501,
 (114, 126): 138,
 (63, 114): 131,
 (63, 126): 72,
 (5, 63): 256,
 (29, 114): 420,
 (29, 126): 328,
 (5, 29): 83,
 (5, 75): 289,
 (5, 79): 289,
 (5, 93): 289,
 (29, 75): 50,
 (29, 79): 50,
 (29, 93): 50,
 (75, 126): 28,
 (79, 126): 28,
 (93, 126): 28,
 (75, 114): 196,
 (79, 114): 196,
 (93, 114): 196,
 (63, 75): 13,
 (63, 79): 13,
 (63, 93): 13,
 (29, 63): 36}

In [ ]:

def save_peptides(path, prot, enzyme, mc=2, charges={1, 2, 3, 4, 5, 6}):
    with open(path, "w") as file:
        writer = csv.DictWriter(file, fieldnames=["peptide", "mods", "mz", "charge"])
        writer.writeheader()
        writer.writerows(
            {
                "peptide": str(p),
                "mods": p.modstr,
                "mz": p.total_mz,
                "charge": p.total_charge,
            }
            for p in Protein(prot).digest(enzyme, charges=charges, maxskip=mc)
            if not p.ismerged
        )


def get_peptides(path, protein, enzyme, mc):
    with open(path, "w") as file:
        peptides_initial = enzyme(protein)
        peptides_skip = {
            (peptides_initial[i][0], peptides_initial[i + s][1], s)
            for s in range(mc + 1)
            for i in range(len(peptides_initial) - s)
        }
        writer = csv.DictWriter(
            file, fieldnames=["peptide", "mc", "position_inclusive"]
        )
        writer.writeheader()
        writer.writerows(
            {"peptide": protein[b:e], "mc": skip, "position_inclusive": f"{b+1}–{e}"}
            for b, e, skip in peptides_skip
        )


# save_peptides(
#     "/Users/eugen/code/bp/bp-code/out/martin/peptides_HAGEN_TRY.csv",
#     MYST,
#     trypsin,
#     mc=7,
#     charges={1},
# )

# for protein, name in [(LYS, "lys"), (BSA, "bsa"), (MYST, "hagen")]:
#     for enzyme, enzyme_name in [(trypsin, "try"), (pepsin, "pep")]:
#         get_peptides(
#             f"../out/martin/peptides_mc7_{name}_{enzyme_name}.csv",
#             protein,
#             enzyme,
#             mc=7,
#         )

In [ ]:
tolerance_ppm_precursor = 100000
multiplier = measurement.binary_multiplier(tolerance_ppm=10)

result = []
peps_with_threshold = [
    (pep, (tolerance_ppm_precursor / 1e6) * pep.total_mz) for pep in peptides
]
for i, m in enumerate(measurements):
    if i % 50 == 0:
        print(f"Done: {i}")

    for pepid, (pep, t) in enumerate(peps_with_threshold):
        if pep.total_charge == m.charge and abs(pep.total_mz - m.peptide_mz) <= t:
            score = m.score_match(pep, multiplier)

            # result.append(
            #     {
            #         "peptide": str(pep),
            #         "is_valid": pep.ismerged or len(pep.peptides[0].seq) > 5,
            #         "score": score,
            #         "mod": pep.modstr,
            #         "scan": m.scan,
            #         "mass_error": pep.total_mz - m.peptide_mz,
            #         "measurement": i,
            #         "pepid": pepid,
            #         "cysteines": pep.cysteines
            #         # "silico_frags": ";".join(str(m) for m in pep.fragment_masses()),
            #         # "frags": ";".join(str(f) for f in m.fragments_mz),
            #         # "frags_intensity": ";".join(str(f) for f in m.fragments_intensity)
            #     }
            # )

# Nechat pokud
# má vysokou pnost
# nebo obsahuje cystein a je kolem něj díra, a jinak má dobré pokrytí
# nebo je krátký a obsahuje cystein, pak na pokrytí nezáleží

# a u těchto se podívat na to, co v tom spektru přebývá?

In [ ]:
tolerance_ppm_precursor = 1000
multiplier = measurement.binary_multiplier(tolerance_ppm=10)

peps_with_threshold = [
    (pep, (tolerance_ppm_precursor / 1e6) * pep.total_mz) for pep in peptides
]

In [ ]:
result = []
tasks = peps_with_threshold[:100]

with tqdm.tqdm(total=len(tasks)) as pbar:

    def get_item(peptide, threshold):
        print("X")
        pbar.update(1)
        for i, m in enumerate(measurements):
            if (
                peptide.total_charge == m.charge
                and abs(pep.total_mz - m.peptide_mz) <= threshold
            ):
                score = m.score_match(peptide, multiplier)

                return {
                    "peptide": str(peptide),
                    "score": score,
                    "mod": peptide.modstr,
                    "scan": m.scan,
                    "mass_error": 1e6
                    * (peptide.total_mz - m.peptide_mz)
                    / peptide.total_mz,
                    "measurement": i,
                    "pepid": pepid,
                    "cysteines": peptide.cysteines,
                }

    def f(x):
        pbar.update()
        return x

    with Pool() as pool:
        for r in pool.imap_unordered(f, tasks):
            print(r)

In [ ]:
df = pd.DataFrame(result)
df.to_csv("../out/lys_analysis_at.csv", index=False)


In [236]:

def show_match(
    msmnt, peptide, multiplier=measurement.binary_multiplier(tolerance_ppm=10)
):
    generated = peptide.fragment_masses

    indices = np.searchsorted(msmnt.masses_padded, generated)
    left = multiplier(generated, msmnt.masses_padded[indices - 1])
    right = multiplier(generated, msmnt.masses_padded[indices])

    matched_intensities = np.maximum(
        left * msmnt.intensities_padded[indices - 1],
        right * msmnt.intensities_padded[indices],
    )

    matched_masses = np.select(
        [
            (left == 0) & (right == 0),
            left * msmnt.intensities_padded[indices - 1]
            > right * msmnt.intensities_padded[indices],
            True,
        ],
        [
            0,
            msmnt.masses_padded[indices - 1],
            msmnt.masses_padded[indices],
        ],
    )

    nonzero = np.nonzero(matched_masses)[0]

    fragments = [peptide.fragment()[i] for i in nonzero]
    intesities_m = matched_intensities[nonzero]

    return {
        "fragments": [
            {
                "name": f.name,
                "mass_theoretical": f.mz,
                "mass_measured": matched_masses[nonzero][i],
                "intensity": round(intesities_m[i], 4),
                "intensity_relative": round(
                    100 * intesities_m[i] / np.sum(msmnt.fragments_intensity), 2
                ),
            }
            for i, f in enumerate(fragments)
        ],
        "score": round(
            100 * np.sum(intesities_m) / np.sum(msmnt.fragments_intensity),
            ndigits=2,
        ),
    }


In [ ]:
m = measurements[3813]
p = [
    p
    for p in peptides
    if str(p) == "KIVSDGNGMNAWVAWR" and p.total_charge == 2 and p.modstr == "[]"
][0]
pprint(show_match(m, p))

In [5]:
source = "../out/190318_LYS_AT_50x_05_scores.pickle"
dicts = []
with open(source, "rb") as file:
    while True:
        try:
            for p in pickle.load(file):
                dicts.append(p)
        except:
            break
df = pd.DataFrame(dicts)
df

,peptide,peptide_id,measurement,measurement_id,measurement_scan,score,prec_error_ppm
0,RHGLDNYRGYSLGNWVCAAK,2,<measurement.PeptideMeasurement object at 0x11...,4453,5767,0.078521,1315.922658
1,RHGLDNYRGYSLGNWVCAAK,2,<measurement.PeptideMeasurement object at 0x11...,4504,5822,0.048617,-9292.491281
2,RHGLDNYRGYSLGNWVCAAK,2,<measurement.PeptideMeasurement object at 0x11...,5627,7007,0.033148,-3862.607785
3,RHGLDNYRGYSLGNWVCAAK,2,<measurement.PeptideMeasurement object at 0x11...,6977,8426,0.078980,9773.471492
4,RHGLDNYRGYSLGNWVCAAK,2,<measurement.PeptideMeasurement object at 0x11...,8043,9545,0.035010,124.050471
...,...,...,...,...,...,...,...
166091,TPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNR...,722197,<measurement.PeptideMeasurement object at 0x6c...,6290,7703,0.063555,-4449.302392
166092,TPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNR...,722197,<measurement.PeptideMeasurement object at 0x6c...,6495,7920,0.050335,-2241.965649
166093,TPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNR...,722197,<measurement.PeptideMeasurement object at 0x6c...,7140,8597,0.043271,-4512.258151
166094,TPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNR...,722198,<measurement.PeptideMeasurement object at 0x6c...,4387,5698,0.089163,-4573.598583


In [55]:

def show_evidence(df):
    evidence = {}

    for peptide, score, err in zip(df.peptide, df.score, df.prec_error_ppm):
        if abs(err) <= 10:
            for c in ((b + 1, e + 1) for b, e in peptide.cysteines):
                current = evidence.setdefault(c, 0)
                evidence[c] = current + score

    return evidence


evidence = show_evidence(df)

Cysteine (6, 127): 63.56315837012556
Cysteine (30, 115): 22.23709135250189
Cysteine (6, 115): 5.6792985082803265
Cysteine (6, 64): 4.8436639296773025
Cysteine (64, 115): 2.9494491960161273
Cysteine (64, 127): 2.301268150823785
Cysteine (30, 127): 1.90940482254393
Cysteine (6, 94): 1.2618185053951843
Cysteine (6, 30): 1.186502645072872
Cysteine (6, 80): 0.9481047386518159
Cysteine (6, 76): 0.6598522364309453
Cysteine (76, 115): 0.3470389215739388
Cysteine (30, 64): 0.23384890946184128
Cysteine (30, 76): 0.1075331707336547
Cysteine (30, 80): 0.1075331707336547
Cysteine (30, 94): 0.1075331707336547


In [124]:
df = df.assign(
    cysteine=[
        (p.cysteines[0][0] + 1, p.cysteines[0][1] + 1)
        if len(p.cysteines) > 0
        else np.NaN
        for p in df.peptide
    ],
    minlen=[np.min([len(p.seq) for p in pep.peptides]) for pep in df.peptide],
)

df.query("abs(prec_error_ppm) <= 25 & minlen > 3").groupby("cysteine").agg(
    {"score": np.sum, "minlen": np.min}
).sort_values("score", ascending=False)

,score,minlen
cysteine,,
"(6, 127)",49.280116,4
"(6, 64)",5.587818,7
"(6, 115)",4.281959,4
"(6, 30)",3.519083,8
"(30, 115)",2.013966,4
"(6, 94)",1.504249,8
"(30, 127)",1.401605,4
"(6, 80)",1.276947,8
"(6, 76)",0.988695,8


In [134]:
df.sort_values("score", ascending=False).groupby("measurement_scan").head(1).query(
    "abs(prec_error_ppm) <= 10000000 & minlen > 3"
).groupby("cysteine").agg({"score": np.sum, "minlen": np.min}).sort_values(
    "score", ascending=False
)

,score,minlen
cysteine,,
"(6, 115)",48.783007,4
"(6, 127)",42.180074,4
"(6, 94)",22.287954,8
"(6, 30)",19.054865,8
"(6, 64)",17.516740,7
"(30, 115)",12.338307,4
"(30, 127)",8.025014,4
"(76, 115)",7.857422,4
"(64, 115)",6.872380,4


In [146]:
df[df.cysteine == (64, 80)].sort_values("score", ascending=False).groupby(
    "measurement_scan"
).head(1).sort_values("score")

,peptide,peptide_id,measurement,measurement_id,measurement_scan,score,prec_error_ppm,cysteine,minlen
134972,WWCNDGRTPGSR+TPGSRNLCNIPCSALLSSDITASVNCAK,482589,<measurement.PeptideMeasurement object at 0x57...,5187,6546,0.012346,-3860.943967,"(64, 80)",12
135449,WWCNDGR+WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIV...,490479,<measurement.PeptideMeasurement object at 0x57...,5195,6554,0.023534,-5227.408229,"(64, 80)",7
134078,NTDGSTDYGILQINSRWWCNDGR+NLCNIPCSALLSSDITASVNCA...,468682,<measurement.PeptideMeasurement object at 0x56...,5110,6465,0.025628,4643.362686,"(64, 80)",23
133672,NTDGSTDYGILQINSRWWCNDGRTPGSR+NLCNIPCSALLSSDITA...,460814,<measurement.PeptideMeasurement object at 0x56...,8436,9957,0.025904,7211.961301,"(64, 80)",23
134540,WWCNDGRTPGSR+TPGSRNLCNIPCSALLSSDITASVNCAKKIVSD...,481981,<measurement.PeptideMeasurement object at 0x57...,6305,7719,0.026627,5412.610004,"(64, 80)",12
...,...,...,...,...,...,...,...,...,...
134559,WWCNDGRTPGSR+NLCNIPCSALLSSDITASVNCAKKIVSDGNGMN...,482242,<measurement.PeptideMeasurement object at 0x57...,5728,7113,0.168685,-3641.999274,"(64, 80)",12
133540,WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKK+WWCNDGRTP...,460255,<measurement.PeptideMeasurement object at 0x56...,8971,10519,0.173146,5100.867086,"(64, 80)",35
134302,WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAK+NLCNIPCSAL...,476742,<measurement.PeptideMeasurement object at 0x57...,5058,6410,0.178758,8325.279828,"(64, 80)",24
133900,NTDGSTDYGILQINSRWWCNDGRTPGSR+WWCNDGRTPGSRNLCNI...,461112,<measurement.PeptideMeasurement object at 0x56...,6805,8245,0.191258,-4207.652594,"(64, 80)",28


In [152]:
d = df.sort_values("score", ascending=False).groupby("measurement_scan").head(1)

d[df.cysteine == (64, 80)].assign(
    err=abs(d.prec_error_ppm), mods=[p.modstr for p in d.peptide]
).sort_values("err")

<ipython-input-152-a7d4c1ca09ad>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d[df.cysteine == (64, 80)].assign(


ValueError: Length of values (4711) does not match length of index (14)

In [221]:
curr = df.assign(
    mods=[p.modstr for p in df.peptide],
    charge=[p.total_charge for p in df.peptide],
    diff=[
        (p.total_mz - m.peptide_mz) * p.total_charge
        for p, m in zip(df.peptide, df.measurement)
    ],
)[
    [
        "WWCNDGR" in str(p) and "NLCNIPCSALLSSDITASVNCAK" in str(p)
        # and not p.ismerged
        for p in df.peptide
    ]
]